<a href="https://colab.research.google.com/github/ushafromwork/Machine-Learning/blob/main/Model_Selection_in_ML_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [5]:
#Importing the models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [7]:
heart_dataset=pd.read_csv('/content/heart.csv')

In [7]:
heart_dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [8]:
heart_dataset.shape

(1025, 14)

In [9]:
heart_dataset.isnull().sum()

,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


In [10]:
heart_dataset['target'].value_counts()

,count
target,
1,526
0,499


In [11]:
x=heart_dataset.drop(columns='target',axis=1)
y=heart_dataset['target']

In [13]:
print(y)

0       0
1       0
2       0
3       0
4       0
       ..
1020    1
1021    0
1022    0
1023    1
1024    0
Name: target, Length: 1025, dtype: int64


#**Model Selection**

In [12]:
x=np.asarray(x)
y=np.asarray(y)

Comparing the models with default hyper parameter values using Cross validation

In [13]:
#list of models
models=[LogisticRegression(max_iter=1000),SVC(kernel='linear'),KNeighborsClassifier(),RandomForestClassifier(random_state=0)]

In [14]:
def compare_models_cross_validation():
  for model in models:
    cv_score=cross_val_score(model,x,y,cv=5)
    mean_accuracy=sum(cv_score)/len(cv_score)
    mean_accuracy=mean_accuracy*100
    mean_accuracy=round(mean_accuracy,2)
    print('Cross validation accuracy for the', model,'=',cv_score)
    print("Accuracy % of the ",model, mean_accuracy)
    print('-------------------------------------')

In [26]:
compare_models_cross_validation()

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Cross validation accuracy for the LogisticRegression(max_iter=1000) = [0.88292683 0.85853659 0.87804878 0.82439024 0.80487805]
Accuracy % of the  LogisticRegression(max_iter=1000) 84.98
-------------------------------------
Cross validation accuracy for the SVC(kernel='linear') = [0.88292683 0.86829268 0.84390244 0.81463415 0.80487805]
Accuracy % of the  SVC(kernel='linear') 84.29
-------------------------------------
Cross validation accuracy for the KNeighborsClassifier() = [0.76585366 0.74634146 0.76097561 0.71219512 0.75121951]
Accuracy % of the  KNeighborsClassifier() 74.73
-------------------------------------
Cross validation accuracy for the RandomForestClassifier(random_state=0) = [1.         1.         0.98536585 1.         0.98536585]
Accuracy % of the  RandomForestClassifier(random_state=0) 99.41
-------------------------------------


**Inference:**

Random Forest Classifier has highest accuracy with default hyper parameter values

Comparing the models with different hyper parameter values using GridSearchCV

In [19]:
models_list=[LogisticRegression(max_iter=10000),SVC(),KNeighborsClassifier(),RandomForestClassifier(random_state=0)]

In [20]:
#Creating a dictionary that contains hyperparameter values for the above mentioned values,
#random forest classifier consists multiple decision trees
model_hyperparameters={
    'log_reg_hyperparameters':{

        'C':[1,5,10,20]
    },

    'svc_hyperparameters': {

        'kernel':['linear','poly','rbf','sigmoid'],
        'C':[1,5,10,20]
    },

    'kNeighbors_hyperparameters':{
        'n_neighbors':[3,5,10]
    },

    'random_hyperparameter':{
        'n_estimators':[10,20,50,100]
    }

}

In [21]:
type(model_hyperparameters)

dict

In [22]:
print(model_hyperparameters.values())

dict_values([{'C': [1, 5, 10, 20]}, {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}, {'n_neighbors': [3, 5, 10]}, {'n_estimators': [10, 20, 50, 100]}])


In [23]:
model_keys=list(model_hyperparameters.keys())

In [24]:
print(model_keys)

['log_reg_hyperparameters', 'svc_hyperparameters', 'kNeighbors_hyperparameters', 'random_hyperparameter']


In [25]:
model_keys[0]

'log_reg_hyperparameters'

In [26]:
model_hyperparameters[model_keys[0]]

{'C': [1, 5, 10, 20]}

#Applying GridSearchCV

In [33]:
def ModelSelection(list_of_models,hyperparameters_dictionary):
    result=[]

    i=0
    for model in list_of_models:
      key=model_keys[i]
      params=hyperparameters_dictionary[key]

      i+=1

      print(model)
      print(params)

      classifier=GridSearchCV(model,params,cv=5)
      #fitting the data to classifier
      classifier.fit(x,y)
      result.append({
          'model used': model,
           'highest score': classifier.best_score_,
           'best hyperparameters': classifier.best_params_
          })

    result_dataframe=pd.DataFrame(result,columns=['model used','highest score','best hyperparameters'])
    return result_dataframe

In [34]:
ModelSelection(models_list,model_hyperparameters)


LogisticRegression(max_iter=10000)
{'C': [1, 5, 10, 20]}
SVC()
{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}
KNeighborsClassifier()
{'n_neighbors': [3, 5, 10]}
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50, 100]}


,model used,highest score,best hyperparameters
0,LogisticRegression(max_iter=10000),0.849756,{'C': 1}
1,SVC(),0.844878,"{'C': 5, 'kernel': 'linear'}"
2,KNeighborsClassifier(),0.890732,{'n_neighbors': 3}
3,RandomForestClassifier(random_state=0),0.997073,{'n_estimators': 20}
